## Setup

### Utils

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 70.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.3 MB/s eta 0:00:00


In [ ]:
from scipy.spatial.distance import cosine

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
from transformers import AutoTokenizer, BertModel
model = BertModel.from_pretrained('bert-base-multilingual-cased', output_hidden_states = True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-cased")

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from Utils import BERT_Embeddings, preprocessing, tokens_evaluation
from sklearn.metrics import classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import numpy as np
import torch
from IPython.display import HTML
from tqdm import tqdm

In [ ]:
from sklearn.metrics import classification_report

### Load training Data

In [ ]:
train_df = pd.read_json("./ArMIS_dataset/ArMIS_train.json", orient='index')
train_df = preprocessing.get_dataset_labels(train_df)

train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement
1,النسويه يعني نصير رجل قولتك وبعدين اذا الوحده ...,0,1.00,0.00,1
2,هراء النسويات عمان والمتستر بتضخيم حالات العنف...,1,0.33,0.67,0
3,بسبب انتشار الفكر النسوي القد ودعوة النسويات ا...,1,0.00,1.00,1
4,وقال تعالى الن الل احا الل تأملي هذة الأيه,0,1.00,0.00,1
5,يعني السعوديات صايعات يتنقلو واحد للثاني,0,0.67,0.33,0
...,...,...,...,...,...
653,رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر اك...,1,0.33,0.67,0
654,تتع مئات الآلاف الفتيات والنساء يوميا جميع أنح...,0,1.00,0.00,1
655,وغيرها الداخل والخارج سافرات ومتعديات حدود الد...,1,0.33,0.67,0
656,والله ياصديقي انا اشفق عليها يسعفها عقلها ولم ...,1,0.00,1.00,1


In [ ]:
# sentences
train_df['sentences'] = train_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
train_df['sentences'] = train_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
train_df['sentences'] = train_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))


In [ ]:
train_df['tokens_lists'] = train_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x), 'ar')
train_df['tokens_lists']= train_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
train_df['tokens_list']= train_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,النسويه يعني نصير رجل قولتك وبعدين اذا الوحده ...,0,1.00,0.00,1,[النسويه يعني نصير رجل قولتك وبعدين اذا الوحده...,"[[النسويه, يعني, نصير, رجل, قولتك, وبعدين, اذا...","[النسويه, يعني, نصير, رجل, قولتك, وبعدين, اذا,..."
2,هراء النسويات عمان والمتستر بتضخيم حالات العنف...,1,0.33,0.67,0,[هراء النسويات عمان والمتستر بتضخيم حالات العن...,"[[هراء, النسويات, عمان, والمتستر, بتضخيم, حالا...","[هراء, النسويات, عمان, والمتستر, بتضخيم, حالات..."
3,بسبب انتشار الفكر النسوي القد ودعوة النسويات ا...,1,0.00,1.00,1,[بسبب انتشار الفكر النسوي القد ودعوة النسويات ...,"[[بسبب, انتشار, الفكر, النسوي, القد, ودعوة, ال...","[بسبب, انتشار, الفكر, النسوي, القد, ودعوة, الن..."
4,وقال تعالى الن الل احا الل تأملي هذة الأيه,0,1.00,0.00,1,[وقال تعالى الن الل احا الل تأملي هذة الأيه],"[[وقال, تعالى, الن, الل, احا, الل, تأملي, هذة,...","[وقال, تعالى, الن, الل, احا, الل, تأملي, هذة, ..."
5,يعني السعوديات صايعات يتنقلو واحد للثاني,0,0.67,0.33,0,[يعني السعوديات صايعات يتنقلو واحد للثاني],"[[يعني, السعوديات, صايعات, يتنقلو, واحد, للثاني]]","[يعني, السعوديات, صايعات, يتنقلو, واحد, للثاني]"
...,...,...,...,...,...,...,...,...
653,رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر اك...,1,0.33,0.67,0,[رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر ا...,"[[رأيج, محترم, جدا, ومنطقي, وطرحج, معقول, اغلب...","[رأيج, محترم, جدا, ومنطقي, وطرحج, معقول, اغلب,..."
654,تتع مئات الآلاف الفتيات والنساء يوميا جميع أنح...,0,1.00,0.00,1,[تتع مئات الآلاف الفتيات والنساء يوميا جميع أن...,"[[تتع, مئات, الآلاف, الفتيات, والنساء, يوميا, ...","[تتع, مئات, الآلاف, الفتيات, والنساء, يوميا, ج..."
655,وغيرها الداخل والخارج سافرات ومتعديات حدود الد...,1,0.33,0.67,0,[وغيرها الداخل والخارج سافرات ومتعديات حدود ال...,"[[وغيرها, الداخل, والخارج, سافرات, ومتعديات, ح...","[وغيرها, الداخل, والخارج, سافرات, ومتعديات, حد..."
656,والله ياصديقي انا اشفق عليها يسعفها عقلها ولم ...,1,0.00,1.00,1,[والله ياصديقي انا اشفق عليها يسعفها عقلها ولم...,"[[والله, ياصديقي, انا, اشفق, عليها, يسعفها, عق...","[والله, ياصديقي, انا, اشفق, عليها, يسعفها, عقل..."


In [ ]:
train_df['original_text']= train_df['tokens_list'].apply(lambda x: ' '.join(x))
train_df['original_text'] = train_df['original_text'].apply(lambda x :x.replace('\u200d', ''))
train_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,النسويه يعني نصير رجل قولتك وبعدين اذا الوحده ...,0,1.00,0.00,1,[النسويه يعني نصير رجل قولتك وبعدين اذا الوحده...,"[[النسويه, يعني, نصير, رجل, قولتك, وبعدين, اذا...","[النسويه, يعني, نصير, رجل, قولتك, وبعدين, اذا,..."
2,هراء النسويات عمان والمتستر بتضخيم حالات العنف...,1,0.33,0.67,0,[هراء النسويات عمان والمتستر بتضخيم حالات العن...,"[[هراء, النسويات, عمان, والمتستر, بتضخيم, حالا...","[هراء, النسويات, عمان, والمتستر, بتضخيم, حالات..."
3,بسبب انتشار الفكر النسوي القد ودعوة النسويات ا...,1,0.00,1.00,1,[بسبب انتشار الفكر النسوي القد ودعوة النسويات ...,"[[بسبب, انتشار, الفكر, النسوي, القد, ودعوة, ال...","[بسبب, انتشار, الفكر, النسوي, القد, ودعوة, الن..."
4,وقال تعالى الن الل احا الل تأملي هذة الأيه,0,1.00,0.00,1,[وقال تعالى الن الل احا الل تأملي هذة الأيه],"[[وقال, تعالى, الن, الل, احا, الل, تأملي, هذة,...","[وقال, تعالى, الن, الل, احا, الل, تأملي, هذة, ..."
5,يعني السعوديات صايعات يتنقلو واحد للثاني,0,0.67,0.33,0,[يعني السعوديات صايعات يتنقلو واحد للثاني],"[[يعني, السعوديات, صايعات, يتنقلو, واحد, للثاني]]","[يعني, السعوديات, صايعات, يتنقلو, واحد, للثاني]"
...,...,...,...,...,...,...,...,...
653,رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر اك...,1,0.33,0.67,0,[رأيج محترم جدا ومنطقي وطرحج معقول اغلب اكدر ا...,"[[رأيج, محترم, جدا, ومنطقي, وطرحج, معقول, اغلب...","[رأيج, محترم, جدا, ومنطقي, وطرحج, معقول, اغلب,..."
654,تتع مئات الآلاف الفتيات والنساء يوميا جميع أنح...,0,1.00,0.00,1,[تتع مئات الآلاف الفتيات والنساء يوميا جميع أن...,"[[تتع, مئات, الآلاف, الفتيات, والنساء, يوميا, ...","[تتع, مئات, الآلاف, الفتيات, والنساء, يوميا, ج..."
655,وغيرها الداخل والخارج سافرات ومتعديات حدود الد...,1,0.33,0.67,0,[وغيرها الداخل والخارج سافرات ومتعديات حدود ال...,"[[وغيرها, الداخل, والخارج, سافرات, ومتعديات, ح...","[وغيرها, الداخل, والخارج, سافرات, ومتعديات, حد..."
656,والله ياصديقي انا اشفق عليها يسعفها عقلها ولم ...,1,0.00,1.00,1,[والله ياصديقي انا اشفق عليها يسعفها عقلها ولم...,"[[والله, ياصديقي, انا, اشفق, عليها, يسعفها, عق...","[والله, ياصديقي, انا, اشفق, عليها, يسعفها, عقل..."


### Load Dev Data

In [ ]:
dev_df = pd.read_json("./ArMIS_dataset/ArMIS_dev.json", orient='index')
dev_df = preprocessing.get_dataset_labels(dev_df)

In [ ]:
# sentences
dev_df['sentences'] = dev_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
dev_df['sentences'] = dev_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))
dev_df['tokens_lists'] = dev_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x), 'ar')
dev_df['tokens_lists']= dev_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
dev_df['tokens_list']= dev_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
dev_df['original_text']= dev_df['tokens_list'].apply(lambda x: ' '.join(x))
dev_df['original_text'] = dev_df['original_text'].apply(lambda x :x.replace('\u200d', ''))
dev_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,الي يقرأ اسئلة المعلمات يستغرب انهم يعملون معن...,0,0.67,0.33,0,[الي يقرأ اسئلة المعلمات يستغرب انهم يعملون مع...,"[[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يع...","[الي, يقرأ, اسئلة, المعلمات, يستغرب, انهم, يعم..."
2,النسويات الفسويات,1,0.00,1.00,1,[النسويات الفسويات],"[[النسويات, الفسويات]]","[النسويات, الفسويات]"
3,سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحاض...,1,0.33,0.67,0,[سؤال للـ بسيط اتوقع تكلمنا التويتر دورات ومحا...,"[[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دور...","[سؤال, للـ, بسيط, اتوقع, تكلمنا, التويتر, دورا..."
4,وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ا...,1,0.33,0.67,0,[وبكذا أوكد استنتاجي بإن النسويات قسمين القسم ...,"[[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين,...","[وبكذا, أوكد, استنتاجي, بإن, النسويات, قسمين, ..."
5,انت قلتي قلتي النسويه امرأة,0,1.00,0.00,1,[انت قلتي قلتي النسويه امرأة],"[[انت, قلتي, قلتي, النسويه, امرأة]]","[انت, قلتي, قلتي, النسويه, امرأة]"
...,...,...,...,...,...,...,...,...
137,الرياضات القتالية حيلة نساء لمواجهة العنف المرأة,0,1.00,0.00,1,[الرياضات القتالية حيلة نساء لمواجهة العنف الم...,"[[الرياضات, القتالية, حيلة, نساء, لمواجهة, الع...","[الرياضات, القتالية, حيلة, نساء, لمواجهة, العن..."
138,احب النوع الفسويات لانها تجيب لحالها المسبه تنسحب,1,0.33,0.67,0,[احب النوع الفسويات لانها تجيب لحالها المسبه ت...,"[[احب, النوع, الفسويات, لانها, تجيب, لحالها, ا...","[احب, النوع, الفسويات, لانها, تجيب, لحالها, ال..."
139,اول شوف اخواتك وبعدين تكلم مكان صايعات عندكم ز...,1,0.33,0.67,0,"[اول شوف اخواتك وبعدين تكلم مكان صايعات, عندكم...","[[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات...","[اول, شوف, اخواتك, وبعدين, تكلم, مكان, صايعات,..."
140,ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب قا...,0,1.00,0.00,1,[ليييش ليش يحطون شيء بالنسويه خير مالقوا سبب ق...,"[[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقو...","[ليييش, ليش, يحطون, شيء, بالنسويه, خير, مالقوا..."


### Load Test Data

In [ ]:
test_df = pd.read_json("./ArMIS_dataset/ArMIS_test.json", orient='index')
test_df = preprocessing.get_dataset_labels(test_df)

In [ ]:
# sentences
test_df['sentences'] = test_df['original_text'].apply(lambda x : preprocessing.split_sentence(x))
test_df['sentences'] = test_df['sentences'].apply(lambda x :preprocessing.adjust_split(x))
test_df['sentences'] = test_df['sentences'].apply(lambda x : preprocessing.apply_lemmatization(x))
test_df['tokens_lists'] = test_df['sentences'].apply(lambda x: preprocessing.sentences_tokenizer(x), 'ar')
test_df['tokens_lists']= test_df['tokens_lists'].apply(lambda x: preprocessing.clear_tokens(x))
test_df['tokens_list']= test_df['tokens_lists'].apply(lambda x: preprocessing.flatten_list(x))

In [ ]:
test_df['original_text']= test_df['tokens_list'].apply(lambda x: ' '.join(x))
test_df['original_text'] = test_df['original_text'].apply(lambda x :x.replace('\u200d', ''))
test_df

,original_text,hard_label,soft_label_0,soft_label_1,disagreement,sentences,tokens_lists,tokens_list
1,اللسان السليط الص راخ الجراءة العنهجية والهمجي...,1,0.00,1.00,1,[اللسان السليط الص راخ الجراءة العنهجية والهمج...,"[[اللسان, السليط, الص, راخ, الجراءة, العنهجية,...","[اللسان, السليط, الص, راخ, الجراءة, العنهجية, ..."
2,قناة الطهر أفضل قناة لمن أراد يحافظ أولاده تخد...,0,0.67,0.33,0,[قناة الطهر أفضل قناة لمن أراد يحافظ أولاده تخ...,"[[قناة, الطهر, أفضل, قناة, لمن, أراد, يحافظ, أ...","[قناة, الطهر, أفضل, قناة, لمن, أراد, يحافظ, أو..."
3,عنف يقال سيبويه أحرقت زوجته كتبه لأنه كان يشتغ...,0,0.67,0.33,0,[عنف يقال سيبويه أحرقت زوجته كتبه لأنه كان يشت...,"[[عنف, يقال, سيبويه, أحرقت, زوجته, كتبه, لأنه,...","[عنف, يقال, سيبويه, أحرقت, زوجته, كتبه, لأنه, ..."
4,ليش لازم شيء نسويه عشان شخص لازم ترى عادي اقوم...,0,1.00,0.00,1,[ليش لازم شيء نسويه عشان شخص لازم ترى عادي اقو...,"[[ليش, لازم, شيء, نسويه, عشان, شخص, لازم, ترى,...","[ليش, لازم, شيء, نسويه, عشان, شخص, لازم, ترى, ..."
5,اساس قارنتوا طاعه الزوج بطاعة الرب عجزت افهم ا...,0,1.00,0.00,1,[اساس قارنتوا طاعه الزوج بطاعة الرب عجزت افهم ...,"[[اساس, قارنتوا, طاعه, الزوج, بطاعة, الرب, عجز...","[اساس, قارنتوا, طاعه, الزوج, بطاعة, الرب, عجزت..."
...,...,...,...,...,...,...,...,...
141,تخيل مطوع قال ناقصات عقل دين,0,1.00,0.00,1,[تخيل مطوع قال ناقصات عقل دين],"[[تخيل, مطوع, قال, ناقصات, عقل, دين]]","[تخيل, مطوع, قال, ناقصات, عقل, دين]"
142,تخيلي اقول الله ياخذ النساء اذا وحدة زعلت ردت ...,0,0.67,0.33,0,[تخيلي اقول الله ياخذ النساء اذا وحدة زعلت ردت...,"[[تخيلي, اقول, الله, ياخذ, النساء, اذا, وحدة, ...","[تخيلي, اقول, الله, ياخذ, النساء, اذا, وحدة, ز..."
143,الله يرحمه يغفرله يجعل مثواه الجنه يارب العالم...,0,1.00,0.00,1,[الله يرحمه يغفرله يجعل مثواه الجنه يارب العال...,"[[الله, يرحمه, يغفرله, يجعل, مثواه, الجنه, يار...","[الله, يرحمه, يغفرله, يجعل, مثواه, الجنه, يارب..."
144,مافي الزمن النبي قال زمن الأزمنة يكون النساء أ...,0,0.67,0.33,0,[مافي الزمن النبي قال زمن الأزمنة يكون النساء ...,"[[مافي, الزمن, النبي, قال, زمن, الأزمنة, يكون,...","[مافي, الزمن, النبي, قال, زمن, الأزمنة, يكون, ..."


## Baseline

In [ ]:
tokens_df = pd.read_csv('./Nuovi_Scores/ArMIS_scores.csv', sep='\t')
tokens_df_10 = tokens_df[tokens_df.occurrences >= 10]

### Threshold estimation from Dev

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = tokens_evaluation.find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
      colors_agreement[indexes[new_word_index]]=0

  #tolgo gli zero:
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

141it [00:00, 292.63it/s]


In [ ]:
best_t = 0
best_f1 = 0
for t in np.arange(round(min(pred_somma)), round(max(pred_somma)), 0.1):
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred_somma], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))

SOMMA 

THRESHOLD: 0.5999999999999996

              precision    recall  f1-score   support

           0       0.49      0.37      0.42        49
           1       0.70      0.79      0.74        92

    accuracy                           0.65       141
   macro avg       0.59      0.58      0.58       141
weighted avg       0.63      0.65      0.63       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_media
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIA 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.45      0.43      0.44        49
           1       0.70      0.72      0.71        92

    accuracy                           0.62       141
   macro avg       0.57      0.57      0.57       141
weighted avg       0.61      0.62      0.62       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_mediana
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIANA 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.46      0.47      0.46        49
           1       0.71      0.71      0.71        92

    accuracy                           0.62       141
   macro avg       0.59      0.59      0.59       141
weighted avg       0.63      0.62      0.62       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: 0.2

              precision    recall  f1-score   support

           0       0.40      0.59      0.48        49
           1       0.71      0.53      0.61        92

    accuracy                           0.55       141
   macro avg       0.56      0.56      0.54       141
weighted avg       0.60      0.55      0.56       141



### Performances on Test

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['tokens_list'], tokens_df_10)

  if 'NA' in colors_agreement:
    indexes = tokens_evaluation.find_NA_indices(colors_agreement)
    for new_word_index in range(len(indexes)) :
       colors_agreement[indexes[new_word_index]]=0


  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

145it [00:00, 406.21it/s]


In [ ]:
print('stima NA: NO')

print('SOMMA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.6) for i in pred_somma] ))

print('\n MEDIA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.3) for i in pred_media]))

print('\n MEDIANA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.3) for i in pred_mediana]))

print('\n ALL GREEN \n')
print(classification_report(test_df['disagreement'], [int(i>=0.2) for i in pred_tutti_verdi]))

stima NA: NO
soglia coordinate: almeno 10 occorrenze
valori non pesati
SOMMA 

              precision    recall  f1-score   support

           0       0.41      0.32      0.36        53
           1       0.65      0.74      0.69        92

    accuracy                           0.59       145
   macro avg       0.53      0.53      0.53       145
weighted avg       0.57      0.59      0.57       145


 MEDIA 

              precision    recall  f1-score   support

           0       0.37      0.36      0.36        53
           1       0.63      0.64      0.64        92

    accuracy                           0.54       145
   macro avg       0.50      0.50      0.50       145
weighted avg       0.54      0.54      0.54       145


 MEDIANA 

              precision    recall  f1-score   support

           0       0.38      0.36      0.37        53
           1       0.64      0.66      0.65        92

    accuracy                           0.55       145
   macro avg       0.51    

In [ ]:
# open file in write mode
with open(r'./Results_NoEstimation/ArMIS_Sum.txt', 'w') as fp:
    for item in pred_somma:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./Results_NoEstimation/ArMIS_Mean.txt', 'w') as fp:
    for item in pred_media:
        # write each item on a new line
        fp.write("%s\n" % item)


with open(r'./Results_NoEstimation/ArMIS_Median.txt', 'w') as fp:
    for item in pred_mediana:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./Results_NoEstimation/ArMIS_Verdi.txt', 'w') as fp:
    for item in pred_tutti_verdi:
        # write each item on a new line
        fp.write("%s\n" % item)

# Mean contextualized
mean of the contextualized vectors to obtain one vector per token

## Context Embeddings for token in Train

In [ ]:
sentences = list(train_df['original_text'])

In [ ]:
from collections import OrderedDict

context_embeddings = []
context_tokens = []

for sentence in tqdm(sentences):
  while sentence and sentence[len(sentence)-1] == '*' :
    sentence = sentence[:len(sentence)-1]
  tokenized_text, list_token_embeddings = BERT_Embeddings.text_to_emb(sentence, tokenizer, model)

  # make ordered dictionary to keep track of the position of each word
  tokens = OrderedDict()

  # loop over tokens in sensitive sentence
  for token in tokenized_text[1:-1]:
    # keep track of position of word and whether it occurs multiple times
    if token in tokens:
      tokens[token] += 1
    else:
      tokens[token] = 1

    # compute the position of the current token
    token_indices = [i for i, t in enumerate(tokenized_text) if t == token]
    current_index = token_indices[tokens[token]-1]

    # get the corresponding embedding
    token_vec = list_token_embeddings[current_index]

    # save values
    context_tokens.append(token)
    context_embeddings.append(token_vec)

100%|██████████| 657/657 [03:43<00:00,  2.94it/s]


In [ ]:
def find_token_indices(list_to_check, token):
    """
    Finds the indices of 'NA' values in a list.

    Args:
        list_to_check (list): List to check for 'NA' values.

    Returns:
        list: List of indices where 'NA' values are found.

    """
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == token:
            indices.append(idx)
    return indices

In [ ]:
def retrieve_elements(lst, indexes):
    return [lst[i] for i in indexes]

In [ ]:
context_embeddings_mean = []
context_tokens_mean = []

for tk in set(context_tokens):
  indexes = find_token_indices(context_tokens, tk)
  context_tokens_mean.append(tk)
  context_embeddings_mean.append(torch.mean(torch.stack(retrieve_elements(context_embeddings, indexes)) , dim=0))#np.mean(list(retrieve_elements(context_embeddings, indexes))))

print(len(context_tokens_mean))

6614


In [ ]:
import gc
del context_tokens
del context_embeddings
gc.collect()

1149

## G-Models

#### Threshold on DEV

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []
threshold=0.8

for _, row in tqdm(dev_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['original_text'].split(), tokens_df_10)

  if 'NA' in colors_agreement:
    similar_words, distances_df, new_words  = find_similar_words(row['original_text'], tokens_evaluation.find_NA_indices(colors_agreement), tokenizer, context_tokens_mean_2,context_embeddings_mean_2, model, tokens_df_10)
    for i in range(0, len(new_words)):
      if list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']):
        stimated_coordinate = np.average(
            list(distances_df.loc[(distances_df['new_token']==new_words[i]) & (distances_df['distance']>=threshold)]['Agreement_coordinate']),
            weights=list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']))

        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=stimated_coordinate
      else: #if there isn't any word above the threshold
        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  while 'NA' in colors_agreement:
    colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0

  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

141it [02:40,  1.14s/it]


In [ ]:
best_t = 0
best_f1 = 0
for t in np.arange(round(min(pred_somma)), round(max(pred_somma)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred_somma], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('SOMMA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred_somma] ))

SOMMA 

THRESHOLD: 2.3

              precision    recall  f1-score   support

           0       0.40      0.35      0.37        49
           1       0.68      0.73      0.70        92

    accuracy                           0.60       141
   macro avg       0.54      0.54      0.54       141
weighted avg       0.58      0.60      0.59       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_media
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIA 

THRESHOLD: 0.4

              precision    recall  f1-score   support

           0       0.41      0.84      0.55        49
           1       0.80      0.35      0.48        92

    accuracy                           0.52       141
   macro avg       0.60      0.59      0.52       141
weighted avg       0.66      0.52      0.51       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_mediana
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print('MEDIANA \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

MEDIANA 

THRESHOLD: 0.3

              precision    recall  f1-score   support

           0       0.50      0.22      0.31        49
           1       0.68      0.88      0.77        92

    accuracy                           0.65       141
   macro avg       0.59      0.55      0.54       141
weighted avg       0.62      0.65      0.61       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] > best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: -0.1

              precision    recall  f1-score   support

           0       0.39      0.29      0.33        49
           1       0.67      0.76      0.71        92

    accuracy                           0.60       141
   macro avg       0.53      0.52      0.52       141
weighted avg       0.57      0.60      0.58       141



In [ ]:
best_t = 0
best_f1 = 0
pred = pred_tutti_verdi
for t in np.arange(round(min(pred)), round(max(pred)), 0.1):
  t = round(t,1)
  report = classification_report(dev_df['disagreement'], [int(i>=t) for i in pred], output_dict=True)
  if report['macro avg']['f1-score'] >= best_f1:
    best_f1 = report['macro avg']['f1-score']
    best_t = t

print(' ALL GREEN \n')
print('THRESHOLD: '+ str(best_t) + '\n')
print(classification_report(dev_df['disagreement'], [int(i>=best_t) for i in pred] ))

 ALL GREEN 

THRESHOLD: -0.1

              precision    recall  f1-score   support

           0       0.39      0.29      0.33        49
           1       0.67      0.76      0.71        92

    accuracy                           0.60       141
   macro avg       0.53      0.52      0.52       141
weighted avg       0.57      0.60      0.58       141



#### Performances on Test

In [ ]:
pred_somma = []
pred_tutti_verdi = []
pred_media = []
pred_mediana = []
threshold=0.8

for _, row in tqdm(test_df.iterrows()):
  colors_agreement, _ = tokens_evaluation.get_all_colors(row['original_text'].split(), tokens_df_10)

  if 'NA' in colors_agreement:
    similar_words, distances_df, new_words  = find_similar_words(row['original_text'], tokens_evaluation.find_NA_indices(colors_agreement), tokenizer, context_tokens_mean_2,context_embeddings_mean_2, model, tokens_df_10)
    for i in range(0, len(new_words)):
      if list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']):
        stimated_coordinate = np.average(
            list(distances_df.loc[(distances_df['new_token']==new_words[i]) & (distances_df['distance']>=threshold)]['Agreement_coordinate']),
            weights=list(distances_df.loc[(distances_df['new_token']==new_words[i])& (distances_df['distance']>=threshold)]['distance']))

        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=stimated_coordinate
      else: #if there isn't any word above the threshold
        colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  while 'NA' in colors_agreement:
    colors_agreement[tokens_evaluation.find_NA_indices(colors_agreement)[0]]=0
  colors_agreement = [i for i in colors_agreement if i != 0]

  if colors_agreement:

    pred_somma.append(sum(colors_agreement))
    pred_media.append(np.mean(colors_agreement))
    pred_mediana.append(np.median(colors_agreement))
    pred_tutti_verdi.append(min(colors_agreement))

  else:
    pred_somma.append(0)
    pred_media.append(0)
    pred_mediana.append(0)
    pred_tutti_verdi.append(0)

145it [02:33,  1.06s/it]


In [ ]:
print('SOMMA \n')
print(classification_report(test_df['disagreement'], [int(i>=2.3) for i in pred_somma] ))

print('\n MEDIA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.4) for i in pred_media]))

print('\n MEDIANA \n')
print(classification_report(test_df['disagreement'], [int(i>=0.3) for i in pred_mediana]))

print('\n ALL GREEN \n')
print(classification_report(test_df['disagreement'], [int(i>=0.1) for i in pred_tutti_verdi]))

stima NA: NO
soglia coordinate: almeno 10 occorrenze
valori non pesati
SOMMA 

              precision    recall  f1-score   support

           0       0.24      0.17      0.20        53
           1       0.59      0.70      0.64        92

    accuracy                           0.50       145
   macro avg       0.42      0.43      0.42       145
weighted avg       0.46      0.50      0.48       145


 MEDIA 

              precision    recall  f1-score   support

           0       0.41      0.79      0.54        53
           1       0.74      0.35      0.47        92

    accuracy                           0.51       145
   macro avg       0.58      0.57      0.51       145
weighted avg       0.62      0.51      0.50       145


 MEDIANA 

              precision    recall  f1-score   support

           0       0.37      0.19      0.25        53
           1       0.64      0.82      0.71        92

    accuracy                           0.59       145
   macro avg       0.50    

In [ ]:
# open file in write mode
with open(r'./ArMIS_G-Sum.txt', 'w') as fp:
    for item in pred_somma:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./ArMIS_G-Mean.txt', 'w') as fp:
    for item in pred_media:
        # write each item on a new line
        fp.write("%s\n" % item)


with open(r'./ArMIS_G-Median.txt', 'w') as fp:
    for item in pred_mediana:
        # write each item on a new line
        fp.write("%s\n" % item)

with open(r'./ArMIS_G-Verdi.txt', 'w') as fp:
    for item in pred_tutti_verdi:
        # write each item on a new line
        fp.write("%s\n" % item)